<a href="https://colab.research.google.com/github/Higgins2718/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/LS_DS_Unit_4_Sprint_Challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Major Neural Network Architectures Challenge
## *Data Science Unit 4 Sprint 3 Challenge*

In this sprint challenge, you'll explore some of the cutting edge of Data Science. This week we studied several famous neural network architectures: 
recurrent neural networks (RNNs), long short-term memory (LSTMs), convolutional neural networks (CNNs), and Generative Adverserial Networks (GANs). In this sprint challenge, you will revisit these models. Remember, we are testing your knowledge of these architectures not your ability to fit a model with high accuracy. 

__*Caution:*__  these approaches can be pretty heavy computationally. All problems were designed so that you should be able to achieve results within at most 5-10 minutes of runtime on Colab or a comparable environment. If something is running longer, doublecheck your approach!

## Challenge Objectives
*You should be able to:*
* <a href="#p1">Part 1</a>: Train a RNN classification model
* <a href="#p2">Part 2</a>: Utilize a pre-trained CNN for objective detection
* <a href="#p3">Part 3</a>: Describe the difference between a discriminator and generator in a GAN
* <a href="#p4">Part 4</a>: Describe yourself as a Data Science and elucidate your vision of AI

<a id="p1"></a>
## Part 1 - RNNs

Use an RNN to fit a multi-class classification model on reuters news articles to distinguish topics of articles. The data is already encoded properly for use in an RNN model. 

Your Tasks: 
- Use Keras to fit a predictive model, classifying news articles into topics. 
- Report your overall score and accuracy

For reference, the [Keras IMDB sentiment classification example](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py) will be useful, as well the RNN code we used in class.

__*Note:*__  Focus on getting a running model, not on maxing accuracy with extreme data size or epoch numbers. Only revisit and push accuracy if you get everything else done!

In [2]:
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 2.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


In [0]:
from tensorflow.keras.datasets import reuters

(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=723812,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

In [65]:
# Demo of encoding

word_index = reuters.get_word_index(path="reuters_word_index.json")

print(f"Iran is encoded as {word_index['iran']} in the data")
print(f"London is encoded as {word_index['london']} in the data")
print("Words are encoded as numbers in our dataset.")

Iran is encoded as 779 in the data
London is encoded as 544 in the data
Words are encoded as numbers in our dataset.


In [0]:
word_index

In [0]:
char_to_int = dict((c, i) for i, c in enumerate(word_index)) # "enumerate" retruns index and value. Convert it to dictionary


In [0]:
test = char_to_int["the"]

In [24]:
test

9876

In [7]:
# TODO - your code!

import numpy as np


num_words = len(word_index) # the number of unique characters

print("num words : ", num_words)


num words :  30979


In [104]:
'''Trains and evaluate a simple MLP
on the Reuters newswire topic classification task.
'''
from __future__ import print_function

import numpy as np
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer

max_words = 1000
batch_size = 32
epochs = 5

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

print('Building model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Loading data...
8982 train sequences
2246 test sequences
46 classes
Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)
Building model...
Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 455s 56ms/step - loss: 2.5472 - acc: 0.3521 - val_loss: 2.4919 - val_acc: 0.3315
Epoch 2/5
8083/8083 [==============================] - 430s 53ms/step - loss: 2.4204 - acc: 0.3540 - val_loss: 2.4756 - val_acc: 0.3315
Epoch 3/5
8083/8083 [==============================] - 432s 53ms/step - loss: 2.4155 - acc: 0.3540 - val_loss: 2.4842 - val_acc: 0.3315
Epoch 4/5
8083/8083 [==============================] - 434s 54ms/step - loss: 2.4143 - acc: 0.3540 - val_loss: 2.4755 - val_acc: 0.3315
Epoch 5/5
2246/2246 [==============================] - 21s 9ms/step
Test score: 2.4264975808501137
Test 

In [105]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=1000)


#text = np.array(['this is just some random, stupid text'])

#pred = tokenizer.sequences_to_matrix(text, mode='binary')
#model.predict(pred)

#prediction = model.predict(np.array(tokenizer.texts_to_sequences(text)))
#print(prediction)
phrase = "the"
tokens = tokenizer.texts_to_matrix([phrase])

model.predict(np.array(tokens))

array([[0.01100585, 0.12609577, 0.01910624, 0.99958634, 0.66809946,
        0.00658533, 0.02209833, 0.00255409, 0.04765695, 0.02246246,
        0.03606024, 0.11583731, 0.00796503, 0.05038804, 0.00591469,
        0.0039089 , 0.15796852, 0.01482773, 0.02734345, 0.15610713,
        0.0939413 , 0.02005816, 0.00278312, 0.00985578, 0.02325255,
        0.03407723, 0.00478861, 0.00389019, 0.01112714, 0.00844097,
        0.01322705, 0.00837284, 0.00974402, 0.00248024, 0.01616853,
        0.00152662, 0.02786419, 0.01012483, 0.00942722, 0.0037736 ,
        0.01590324, 0.01200061, 0.0058022 , 0.00322032, 0.00384012,
        0.00426057]], dtype=float32)

In [0]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm


def create_model():
	 # create model
  model = Sequential()
  model.add(Embedding(max_features, 128))
  model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(num_classes, activation='sigmoid'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load dataset
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [2, 3]


param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(x_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print("%f (%f) with: %r" % (mean, stdev, param))

8982 train sequences
2246 test sequences
46 classes
Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)
Convert class vector to binary class matrix (for use with categorical_crossentropy)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Conclusion - RNN runs, and gives pretty decent improvement over a naive model. To *really* improve the model, more playing with parameters would help. Also - RNN may well not be the best approach here, but it is at least a valid one.

<a id="p2"></a>
## Part 2- CNNs

### Find the Frog

Time to play "find the frog!" Use Keras and ResNet50 (pre-trained) to detect which of the following images contain frogs:

<img align="left" src="https://d3i6fh83elv35t.cloudfront.net/newshour/app/uploads/2017/03/GettyImages-654745934-1024x687.jpg" width=400>


In [34]:
!pip install google_images_download

     |████████████████████████████████| 911kB 4.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/1f/28/ad/f56e7061e1d2a9a1affe2f9c649c2570cb9198dd24ede0bbab
Successfully built google-images-download


In [63]:
from google_images_download import google_images_download

response = google_images_download.googleimagesdownload()
arguments = {"keywords": "animal pond", "limit": 15, "print_urls": True}
absolute_image_paths = response.download(arguments)


Item no.: 1 --> Item name = animal pond
Evaluating...
Starting Download...
Image URL: https://www.enchantedlearning.com/pgifs/Pondanimals.GIF
Completed Image ====> 1.Pondanimals.GIF
Image URL: https://i.ytimg.com/vi/NCbu0TND9vE/hqdefault.jpg
Completed Image ====> 2.hqdefault.jpg
Image URL: https://pklifescience.com/staticfiles/articles/images/PKLS4116_inline.png
Completed Image ====> 3.PKLS4116_inline.png
Image URL: https://get.pxhere.com/photo/water-animal-pond-wildlife-mammal-fish-eat-fauna-whiskers-vertebrate-otter-mink-marmot-sea-otter-mustelidae-1383482.jpg
Completed Image ====> 4.water-animal-pond-wildlife-mammal-fish-eat-fauna-whiskers-vertebrate-otter-mink-marmot-sea-otter-mustelidae-1383482.jpg
Image URL: https://pklifescience.com/staticfiles/articles/images/PKLS4116.png
Completed Image ====> 5.PKLS4116.png
Image URL: https://i.pinimg.com/originals/57/5c/5b/575c5b5c441e27ff04eb50571ee30127.jpg
Completed Image ====> 6.575c5b5c441e27ff04eb50571ee30127.jpg
Image URL: https://cdn

At time of writing at least a few do, but since the Internet changes - it is possible your 5 won't. You can easily verify yourself, and (once you have working code) increase the number of images you pull to be more sure of getting a frog. Your goal is to validly run ResNet50 on the input images - don't worry about tuning or improving the model.

*Hint* - ResNet 50 doesn't just return "frog". The three labels it has for frogs are: `bullfrog, tree frog, tailed frog`

*Stretch goal* - also check for fish.

In [0]:
# TODO - your code!import numpy as np

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import requests

def process_img_path(img_path):
  return image.load_img(img_path, target_size=(224, 224))

def img_contains_frog(img):
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  model = ResNet50(weights='imagenet')
  features = model.predict(x)
  results = decode_predictions(features, top=3)[0]
  print(results)
  for entry in results:
    if entry[1] == 'bullfrog' or entry[1] == 'tree frog' or entry[1] == 'tailed frog':
      return entry[2]
  return 0.0

In [64]:
for path in absolute_image_paths[0]['animal pond']:
  img_contains_frog(process_img_path(path))

[('n03598930', 'jigsaw_puzzle', 0.8680324), ('n06359193', 'web_site', 0.064099774), ('n02834397', 'bib', 0.021264149)]
[('n01443537', 'goldfish', 0.8495884), ('n01631663', 'eft', 0.067601345), ('n02536864', 'coho', 0.03516318)]
[('n04243546', 'slot', 0.8712438), ('n04476259', 'tray', 0.04993648), ('n03908618', 'pencil_box', 0.023072599)]
[('n02442845', 'mink', 0.30976573), ('n02363005', 'beaver', 0.2339894), ('n02361337', 'marmot', 0.20796825)]
[('n03485794', 'handkerchief', 0.88227284), ('n02834397', 'bib', 0.02268081), ('n03291819', 'envelope', 0.020095214)]
[('n02116738', 'African_hunting_dog', 0.610158), ('n02105162', 'malinois', 0.19866747), ('n02114712', 'red_wolf', 0.051152844)]
[('n01737021', 'water_snake', 0.30730632), ('n01641577', 'bullfrog', 0.26061353), ('n04275548', 'spider_web', 0.11343399)]
[('n06359193', 'web_site', 0.5092322), ('n02840245', 'binder', 0.10986997), ('n04209239', 'shower_curtain', 0.100119166)]
[('n01698640', 'American_alligator', 0.96394145), ('n0169745

In [0]:
# TODO - your code!import numpy as np

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import requests

def process_img_path(img_path):
  return image.load_img(img_path, target_size=(224, 224))

def img_contains_fish(img):
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  model = ResNet50(weights='imagenet')
  features = model.predict(x)
  results = decode_predictions(features, top=3)[0]
  print(results)
  for entry in results:
    if entry[1].contains('fish'):
      return entry[2]
  return 0.0

<a id="p3"></a>
## Part 3 - Generative Adverserial Networks (GANS)

Describe the difference between a discriminator and generator in a GAN in your own words.

The job of the generator is to generate an image that looks real. The job of the discriminator is to give a probability that the generated image is real.

A well trained GAN finds the generator and discriminator in a Nash equilibrium.

<a id="p4"></a>
## Part 4 - More...

Answer the following questions, with a target audience of a fellow Data Scientist:

- What do you consider your strongest area, as a Data Scientist?

I've been told several times that I'm good at explaining technical things to non-technical people. This is a good skill for a data scientist, as we will often be required to explain analyses of data to non-technical people in a non-technical but detailed manner. 

- What area of Data Science would you most like to learn more about, and why?

I'd like to learn more about dealing with "big data"—particularly when that data is unstructured. Data in the real world is often messy, but valuable. I think my skills need sharpening in this area. Thankfully, we'll be having more instruction on this.

- Where do you think Data Science will be in 5 years?

It looks as though automl will be very effective in five years or so. Google is making very nice progress here. I expect that more people will have a decent understanding of what exactly it is that data scientists do and when to hire them. Perhaps at this point we will see more technical advancements? After I have more experience "in the field" the answer to this question should be clearer to me. 

- What are the treats posed by AI to our society?

Job displacement is the most immediate threat. One could easily imagine mass outrage leading to politicians campaigning on AI-regulation. If we don't accidentally create harmful AGI and manage to create it in a manner that allows it to augment human intelligence, society will fundamentally change on a massive scale. This is increasingly a bad thing. However, this is a distant threat and I rarely think of it. Too many unknowns.

- How do you think we can counteract those threats? 

Regulation is a line of dominoes and China would be the only winner in this case. Don't let the government have any say in who gets to develop AI. Focus more on AI-security startups. 

- Do you think achieving General Artifical Intelligence is ever possible?

Yes, but probably not before 2050. There's not a strong commercial incentive behind AGI research right now and consciousnesss seems to become more mystifying the more we learn about it.

A few sentences per answer is fine - only elaborate if time allows.

## Congratulations! 

Thank you for your hard work, and congratulations! You've learned a lot, and you should proudly call yourself a Data Scientist.


In [0]:
from IPython.display import HTML

HTML("""<iframe src="https://giphy.com/embed/26xivLqkv86uJzqWk" width="480" height="270" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/mumm-champagne-saber-26xivLqkv86uJzqWk">via GIPHY</a></p>""")